In [2]:
import cv2
import mediapipe as mp
import pyautogui
import tkinter as tk
from tkinter import messagebox
import threading
import time
import numpy as np

# Отключение функции безопасности PyAutoGUI
pyautogui.FAILSAFE = False

# Инициализация Tkinter
root = tk.Tk()
root.title("Virtual Keyboard")
root.geometry("300x100")

# Флаг для завершения программы
exit_program = False
camera_index = 0  # Начнем с задней камеры
cap = cv2.VideoCapture(camera_index)

def exit_app():
    global exit_program
    exit_program = True
    messagebox.showinfo("Exit", "Program will exit after releasing resources.")
    root.quit()  # Закрытие окна Tkinter

def switch_camera():
    global camera_index, cap
    camera_index = 1 - camera_index  # Переключение между 0 и 1
    cap.release()
    cap = cv2.VideoCapture(camera_index)
    if not cap.isOpened():
        messagebox.showerror("Error", "Cannot switch to the selected camera.")
        camera_index = 1 - camera_index  # Переключение обратно
        cap = cv2.VideoCapture(camera_index)

exit_button = tk.Button(root, text="Exit", command=exit_app)
exit_button.pack(pady=20)
switch_camera_button = tk.Button(root, text="Switch Camera", command=switch_camera)
switch_camera_button.pack(pady=20)

# Инициализация модулей для распознавания жестов
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.7, min_tracking_confidence=0.7)

# Определение клавиатуры
keyboard = [
    ["Q", "W", "E", "R", "T", "Y", "U", "I", "O", "P"],
    ["A", "S", "D", "F", "G", "H", "J", "K", "L"],
    ["Z", "X", "C", "V", "B", "N", "M"]
]

key_width = 60
key_height = 60
margin = 10
last_pressed_key = None
key_press_time = 0
key_press_delay = 0.5  # Задержка между нажатиями одной и той же клавиши

typed_text = ""  # Хранение напечатанного текста

def draw_keyboard(image, pts1, pts2):
    for row_idx, row in enumerate(keyboard):
        for col_idx, key in enumerate(row):
            x1, y1 = pts1[row_idx][col_idx]
            x2, y2 = pts2[row_idx][col_idx]
            pts = np.float32([[x1, y1], [x2, y1], [x2, y2], [x1, y2]])
            cv2.fillConvexPoly(image, pts.astype(int), (255, 255, 255))
            cx, cy = int((x1 + x2) / 2), int((y1 + y2) / 2)
            cv2.putText(image, key, (cx - 10, cy + 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)

def get_key_at_position(x, y, pts2):
    for row_idx, row in enumerate(keyboard):
        for col_idx, key in enumerate(row):
            x1, y1 = pts2[row_idx][col_idx]
            x2, y2 = x1 + key_width, y1 + key_height
            pts = np.float32([[x1, y1], [x2, y1], [x2, y2], [x1, y2]])
            if cv2.pointPolygonTest(pts, (x, y), False) >= 0:
                return key
    return None

def draw_axes(image, origin, axis_length=100):
    origin = tuple(map(int, origin))
    x_axis = (origin[0] + axis_length, origin[1])
    y_axis = (origin[0], origin[1] - axis_length)
    z_axis = (origin[0] + int(axis_length * 0.7), origin[1] - int(axis_length * 0.7))
    
    cv2.line(image, origin, x_axis, (0, 0, 255), 2)  # X-axis in red
    cv2.line(image, origin, y_axis, (0, 255, 0), 2)  # Y-axis in green
    cv2.line(image, origin, z_axis, (255, 0, 0), 2)  # Z-axis in blue

    cv2.putText(image, "X", x_axis, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
    cv2.putText(image, "Y", y_axis, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    cv2.putText(image, "Z", z_axis, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

def video_loop():
    global exit_program, last_pressed_key, key_press_time, typed_text, cap
    screen_width, screen_height = pyautogui.size()

    while not exit_program:
        if not cap.isOpened():
            cap = cv2.VideoCapture(camera_index)

        ret, frame = cap.read()
        if not ret:
            continue

        # Отразить изображение по горизонтали
        frame = cv2.flip(frame, 1)
        
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(frame_rgb)

        h, w, _ = frame.shape

        # Определение координат для проекции клавиатуры
        start_x = w // 4
        start_y = h - (len(keyboard) * (key_height + margin)) - margin
        pts1 = [[(start_x + col * (key_width + margin), start_y + row * (key_height + margin)) for col in range(len(keyboard[0]))] for row in range(len(keyboard))]
        pts2 = [[(x, y + 20) for (x, y) in row] for row in pts1]

        draw_keyboard(frame, pts1, pts2)
        draw_axes(frame, (start_x, start_y))

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                
                # Получаем координаты кончика указательного пальца
                index_finger_tip = hand_landmarks.landmark[8]
                cx, cy = int(index_finger_tip.x * w), int(index_finger_tip.y * h)
                
                # Определяем, на какую клавишу нажали
                key = get_key_at_position(cx, cy, pts2)
                if key and (key != last_pressed_key or time.time() - key_press_time > key_press_delay):
                    pyautogui.press(key.lower())
                    typed_text += key
                    last_pressed_key = key
                    key_press_time = time.time()
                
                # Рисуем курсор (круг) на кончике указательного пальца
                cv2.circle(frame, (cx, cy), 10, (0, 255, 0), cv2.FILLED)

        # Отображаем напечатанный текст на изображении
        cv2.putText(frame, typed_text, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

        cv2.imshow('Virtual Keyboard', frame)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Запуск видео потока в отдельном потоке
video_thread = threading.Thread(target=video_loop)
video_thread.start()

# Запуск Tkinter основного цикла
root.mainloop()


C:\Users\tepli\AppData\Roaming\Python\Python312\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
